In [6]:
import os
import sqlite3
from datetime import date, timedelta
import random



BACKEND_DIR = '/Users/dilshantharushika/Desktop/laptop agent/backend' 
DB_NAME = 'laptops_dynamic.db'
DB_PATH = os.path.join(BACKEND_DIR, DB_NAME)


os.makedirs(BACKEND_DIR, exist_ok=True)


LAPTOP_SKUS = [
    "ThinkPad E14 Gen 5 (Intel)",
    "Lenovo ThinkPad E14 Gen 5 (AMD)", 
    "HP ProBook 450 G10 — Datasheet",
    "HP ProBook 440 14 inch G11 Notebook PC"
]


LAPTOP_DETAILS = {
    "ThinkPad E14 Gen 5 (Intel)": ("Lenovo", "ThinkPad E14 Gen 5 (Intel)"),
    "Lenovo ThinkPad E14 Gen 5 (AMD)": ("Lenovo", "ThinkPad E14 Gen 5 (AMD)"), 
    "HP ProBook 450 G10 — Datasheet": ("HP", "ProBook 450 G10"),
    "HP ProBook 440 14 inch G11 Notebook PC": ("HP", "ProBook 440 G11")
}


print("Configuration loaded.")
print(f"Database will be created/managed at: {os.path.abspath(DB_PATH)}")
print(f"Laptop SKUs defined (Updated): {LAPTOP_SKUS}")

Configuration loaded.
Database will be created/managed at: /Users/dilshantharushika/Desktop/laptop agent/backend/laptops_dynamic.db
Laptop SKUs defined (Updated): ['ThinkPad E14 Gen 5 (Intel)', 'Lenovo ThinkPad E14 Gen 5 (AMD)', 'HP ProBook 450 G10 — Datasheet', 'HP ProBook 440 14 inch G11 Notebook PC']


In [7]:

def create_database():
    """Connects to the DB, drops old tables, and creates new ones."""
    print(f"Connecting to database at {DB_PATH}...")
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    print("Dropping existing tables (if any)...")
    cursor.execute("DROP TABLE IF EXISTS Laptop")
    cursor.execute("DROP TABLE IF EXISTS PriceHistory")
    cursor.execute("DROP TABLE IF EXISTS Review")
    cursor.execute("DROP TABLE IF EXISTS QuestionAnswer") 

    print("Creating new tables")
 
    cursor.execute('''
    CREATE TABLE Laptop (
        sku TEXT PRIMARY KEY,
        brand TEXT NOT NULL,
        model_name TEXT NOT NULL,
        currency TEXT DEFAULT 'LKR',      -- Added Currency
        availability TEXT DEFAULT 'Check Website', -- Added Availability
        shipping_eta TEXT DEFAULT 'Check Website', -- Added Shipping ETA
        review_count INTEGER DEFAULT 0,  -- Added Review Count
        average_rating REAL DEFAULT 0.0 -- Added Average Rating
    )
    ''')


    cursor.execute('''
    CREATE TABLE PriceHistory (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        laptop_sku TEXT NOT NULL,
        price REAL NOT NULL,
        date TEXT NOT NULL, -- Storing date as ISO format string (YYYY-MM-DD)
        vendor_name TEXT,
        promo_badges TEXT,              -- Added Promo Badges (e.g., "Sale, Free Shipping")
        FOREIGN KEY (laptop_sku) REFERENCES Laptop (sku)
    )
    ''')


    cursor.execute('''
    CREATE TABLE Review (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        laptop_sku TEXT NOT NULL,
        rating INTEGER NOT NULL CHECK(rating >= 1 AND rating <= 5),
        review_text TEXT,
        date TEXT NOT NULL, -- Storing date as ISO format string (YYYY-MM-DD)
        source TEXT,
        FOREIGN KEY (laptop_sku) REFERENCES Laptop (sku)
    )
    ''')


    cursor.execute('''
    CREATE TABLE QuestionAnswer (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        laptop_sku TEXT NOT NULL,
        question_text TEXT NOT NULL,
        answer_text TEXT,
        date TEXT NOT NULL, -- Storing date as ISO format string (YYYY-MM-DD)
        source TEXT,
        FOREIGN KEY (laptop_sku) REFERENCES Laptop (sku)
    )
    ''')

    conn.commit() 
    conn.close()
    print("Database tables created successfully with updated schema.")


print("create_database() function defined.")

create_database() function defined.


In [3]:

def seed_data():
    """Populates the database tables with sample data."""
    print("Seeding data...")
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    today = date.today() 


    print("Seeding Laptop table...")
    for sku in LAPTOP_SKUS:
        if sku in LAPTOP_DETAILS:
            brand, model_name = LAPTOP_DETAILS[sku]
            cursor.execute("""
                INSERT INTO Laptop (sku, brand, model_name, currency, availability, shipping_eta, review_count, average_rating) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                (sku, brand, model_name, 'LKR', 'In Stock', 'Ships in 3-5 days', random.randint(5, 50), round(random.uniform(3.5, 4.8), 1))
            )
        else: 
            print(f"Warning: No details found for SKU: {sku}. Using placeholder.")
            cursor.execute("""
                INSERT INTO Laptop (sku, brand, model_name, currency, availability, shipping_eta, review_count, average_rating) 
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
                (sku, "Unknown", sku, 'LKR', 'Unknown', 'Unknown', 0, 0.0) 
            )


    print("Seeding PriceHistory table...")
    vendors = ["Official Store", "Partner Reseller A", "Partner Reseller B"]
    promos = ["None", "Sale", "Free Shipping", "Bundle Offer", "None", "None"] 
    for sku in LAPTOP_SKUS:
        base_price = random.uniform(300000.0, 600000.0) 
        for i in range(5): 
            price_date = today - timedelta(days=random.randint(0, 60)) 
            price = base_price * random.uniform(0.98, 1.03) 
            vendor = random.choice(vendors)
            promo = random.choice(promos)
            cursor.execute(
                "INSERT INTO PriceHistory (laptop_sku, price, date, vendor_name, promo_badges) VALUES (?, ?, ?, ?, ?)",
                (sku, round(price, 2), price_date.isoformat(), vendor, promo)
            )


    print("Seeding Review table...")
    review_sources = ["Customer Review", "Tech Site", "Forum"]
    sample_reviews = [
        "Good for office work.", "Battery doesn't last a full day.", "Very sturdy design.",
        "Display is clear and bright.", "Typing experience is excellent.", "A bit bulky for travel.", "Fast boot up time.",
        "Decent value for money.", "Trackpad is responsive.", "Handles basic programming tasks well."
    ]
    for sku in LAPTOP_SKUS:
        for i in range(3): 
            review_date = today - timedelta(days=random.randint(0, 180)) 
            rating = random.randint(3, 5)
            review_text = random.choice(sample_reviews)
            source = random.choice(review_sources)
            cursor.execute(
                "INSERT INTO Review (laptop_sku, rating, review_text, date, source) VALUES (?, ?, ?, ?, ?)",
                (sku, rating, review_text, review_date.isoformat(), source)
            )

    
    print("Seeding QuestionAnswer table...")
    sample_questions = [
        "Can I upgrade the RAM?", "Does it have a backlit keyboard?", "What is the warranty period?",
        "Is the webcam good for video calls?", "Does it support external monitors?"
    ]
    sample_answers = [ 
        "Yes, RAM is typically upgradeable on this model series, check specific config.", 
        "Backlit keyboard is available on select configurations.", 
        "Usually comes with a 1-year standard warranty, extensions available.",
        "The FHD webcam option provides good quality for video calls.", 
        "Yes, supports external displays via HDMI and/or USB-C DisplayPort."
    ]
    for sku in LAPTOP_SKUS:
        for i in range(2): 
            qa_date = today - timedelta(days=random.randint(0, 90)) 
            question = random.choice(sample_questions)
            answer = random.choice(sample_answers) 
            source = "Official Q&A / Forum"
            cursor.execute(
                "INSERT INTO QuestionAnswer (laptop_sku, question_text, answer_text, date, source) VALUES (?, ?, ?, ?, ?)",
                (sku, question, answer, qa_date.isoformat(), source)
            )

    conn.commit() 
    conn.close()
    print("Data seeded successfully.")


print("seed_data() function defined.")

seed_data() function defined.


In [8]:

if os.path.exists(DB_PATH):
   print(f"Database file '{DB_PATH}' already exists. Deleting to recreate...")
   os.remove(DB_PATH)
else:
    print(f"Database file '{DB_PATH}' not found. Will create a new one.")


create_database() 
seed_data()

print(" Database setup and seeding complete!")
print(f"Database file created/updated at: {os.path.abspath(DB_PATH)}")

Database file '/Users/dilshantharushika/Desktop/laptop agent/backend/laptops_dynamic.db' already exists. Deleting to recreate...
Connecting to database at /Users/dilshantharushika/Desktop/laptop agent/backend/laptops_dynamic.db...
Dropping existing tables (if any)...
Creating new tables
Database tables created successfully with updated schema.
Seeding data...
Seeding Laptop table...
Seeding PriceHistory table...
Seeding Review table...
Seeding QuestionAnswer table...
Data seeded successfully.
 Database setup and seeding complete!
Database file created/updated at: /Users/dilshantharushika/Desktop/laptop agent/backend/laptops_dynamic.db


testing the db

In [9]:

print(f"Connecting to {DB_PATH} to verify data...")
conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row 
cursor = conn.cursor()

print("\n--- Sample Data from 'Laptop' Table ---")
cursor.execute("SELECT * FROM Laptop LIMIT 2") 
rows = cursor.fetchall()
for row in rows:
    print(dict(row)) 

print("\n--- Sample Data from 'PriceHistory' Table ---")
cursor.execute("SELECT * FROM PriceHistory ORDER BY date DESC LIMIT 3") 
rows = cursor.fetchall()
for row in rows:
    print(dict(row))

print("\n--- Sample Data from 'Review' Table ---")
cursor.execute("SELECT laptop_sku, rating, review_text FROM Review LIMIT 3") 
rows = cursor.fetchall()
for row in rows:
    print(dict(row))

print("\n--- Sample Data from 'QuestionAnswer' Table ---")
cursor.execute("SELECT laptop_sku, question_text FROM QuestionAnswer LIMIT 2") 
rows = cursor.fetchall()
for row in rows:
    print(dict(row))

conn.close()
print("\nVerification complete. Database closed.")

Connecting to /Users/dilshantharushika/Desktop/laptop agent/backend/laptops_dynamic.db to verify data...

--- Sample Data from 'Laptop' Table ---
{'sku': 'ThinkPad E14 Gen 5 (Intel)', 'brand': 'Lenovo', 'model_name': 'ThinkPad E14 Gen 5 (Intel)', 'currency': 'LKR', 'availability': 'In Stock', 'shipping_eta': 'Ships in 3-5 days', 'review_count': 10, 'average_rating': 4.8}
{'sku': 'Lenovo ThinkPad E14 Gen 5 (AMD)', 'brand': 'Lenovo', 'model_name': 'ThinkPad E14 Gen 5 (AMD)', 'currency': 'LKR', 'availability': 'In Stock', 'shipping_eta': 'Ships in 3-5 days', 'review_count': 8, 'average_rating': 3.8}

--- Sample Data from 'PriceHistory' Table ---
{'id': 6, 'laptop_sku': 'Lenovo ThinkPad E14 Gen 5 (AMD)', 'price': 398284.84, 'date': '2025-10-20', 'vendor_name': 'Partner Reseller B', 'promo_badges': 'Sale'}
{'id': 5, 'laptop_sku': 'ThinkPad E14 Gen 5 (Intel)', 'price': 369896.71, 'date': '2025-10-15', 'vendor_name': 'Official Store', 'promo_badges': 'None'}
{'id': 20, 'laptop_sku': 'HP ProBo